# Итоговый проект по курсу Автоматическая Обработка Естественного Языка
#### Выполнили: Женя Егорова, Таня Казакова, Ксюша Кошкина, Настя Чижикова

Корпус текстов собран из постов в пабликах Вконтакте, посвященных феминизму. Тексты выкачаны с помощью VK Api, на начальном этапе взято больше 100 постов длиной не менее 100 слов.

Паблики: https://vk.com/socfem, https://vk.com/radfemtranslations

In [ ]:
pip install natasha

In [ ]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)
import re

In [ ]:
pip install pymorphy2

In [ ]:
pip install nltk

In [ ]:
pip install razdel

In [ ]:
pip install pymorphy2

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import pandas as pd
from razdel import tokenize, sentenize
from string import punctuation
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
import re
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Код для выкачивания постов vk. Вам нужно обновить токен. Или пропустить эти шаги и воспользоваться скачанными из файла sentences.tsv.

In [ ]:
TOKEN = '2030e8202030e8202030e820cf2041e16f220302030e8207ea4b43071db099fb761cad2'
groups = [-182023900, -46661081]

In [ ]:
posts = []
for group_id in groups:
  data = requests.get(
      'https://api.vk.com/method/wall.get', 
       params={
            "owner_id": group_id,
            "count": 100,
            "v":"5.92",
            "access_token": TOKEN
            }
      ).json()
  group_posts = data['response']['items']

  '''Метаинформация для каждого поста: 
  паблик, дата публикации и id поста, чтобы не потерять впоследствии
  отдельные предложения
  '''

  for p in group_posts:
    if len(p['text'].split()) > 99:
      post_url = f"https://vk.com/public{group_id * -1}?w=wall{group_id}_{p['id']}"
      post_text = p['text']
      date = str(datetime.fromtimestamp(p['date']))

      sentences_list = [_.text for _ in list(sentenize(post_text)) if not _.text.startswith('#') ]
      for sent in sentences_list:
        posts.append((post_url, date, sent.replace('\n', ' ')))  

In [ ]:
len(posts)

3468

Сохраняем в tsv-файл (вам не надо)

In [ ]:
'''from google.colab import files

with open('sentences.tsv', 'w', encoding='utf-8') as f:
  for p in posts:
    f.write('\t'.join(p) + '\n')

files.download('sentences.tsv')'''

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Вот это вам надо

In [ ]:
sent_df = pd.read_csv('https://raw.githubusercontent.com/nastyachizhikova/NLP_3rdyear/main/sentences.tsv', sep='\t', header=None, names=['URL', 'Date', 'Sentence']) 

### Предобработка предложений

In [ ]:
def preprocess(sent):
  tokens = [_.text.strip('-') for _ in list(tokenize(str(sent).lower())) if not re.search('[^а-яА-ЯёЁ\-]', _.text)]
  return ' '.join(tokens)

In [ ]:
sent_df['Clean_sentence'] = sent_df['Sentence'].apply(preprocess)

In [ ]:
sent_df

,URL,Date,Sentence,Clean_sentence
0,https://vk.com/public182023900?w=wall-18202390...,2020-10-21 13:36:00,«В июне этого года девушка по имени Киара опуб...,в июне этого года девушка по имени киара опубл...
1,https://vk.com/public182023900?w=wall-18202390...,2020-10-21 13:36:00,Далее в видео появляется надпись: «Решила тут ...,далее в видео появляется надпись решила тут по...
2,https://vk.com/public182023900?w=wall-18202390...,2020-10-21 13:36:00,В конце видео Киара показывает пальцами знак ...,в конце видео киара показывает пальцами знак п...
3,https://vk.com/public182023900?w=wall-18202390...,2020-10-21 13:36:00,На момент публикации этой статьи у поста 5.7 м...,на момент публикации этой статьи у поста милли...
4,https://vk.com/public182023900?w=wall-18202390...,2020-10-21 13:36:00,"Фильм, упоминаемый в этом видео, — это польски...",фильм упоминаемый в этом видео это польский эр...
...,...,...,...,...
3463,https://vk.com/public46661081?w=wall-46661081_...,2020-09-23 17:30:02,Плюс непосредственно на площадке были еще три ...,плюс непосредственно на площадке были еще три ...
3464,https://vk.com/public46661081?w=wall-46661081_...,2020-09-23 17:30:02,Победа на всероссийском этапе дает возможность...,победа на всероссийском этапе дает возможность...
3465,https://vk.com/public46661081?w=wall-46661081_...,2020-09-23 17:30:02,Что касается профессиональной траектории Елены...,что касается профессиональной траектории елены...
3466,https://vk.com/public46661081?w=wall-46661081_...,2020-09-23 17:30:02,"Например, систем «Умный дом» —Елена монтирует ...",например систем умный дом елена монтирует обор...


Датафрейм униграмм

In [ ]:
def get_pos_lem(word):
  analiz = morph.parse(word)
  return analiz[0].normal_form, analiz[0].tag.POS

In [ ]:
from tqdm.auto import tqdm

In [ ]:
uni_df = pd.DataFrame(columns=['Token', 'Lemma', 'POS', 'Sent_ids'])

for sent_id, sent in tqdm(zip(sent_df.index, sent_df.Clean_sentence)):
    for word in sent.split():
        if word not in uni_df.Token.values:
          lem, pos = get_pos_lem(word) 
          uni_df = uni_df.append({'Token': word,
                          'Lemma': lem,
                          'POS': str(pos),
                          'Sent_ids': str(sent_id)
                          }, ignore_index=True)
        else:
          #ids = uni_df[uni_df.Token == word].Sent_ids.values.tolist()
          uni_df.loc[uni_df.Token == word, 'Sent_ids'] += f' {sent_id}'

In [ ]:
def fix_ids(str_list):
  return [int(i) for i in list(set(str_list.split()))]

In [ ]:
uni_df['Ids'] = uni_df['Sent_ids'].apply(fix_ids)


In [ ]:
uni_df

,Token,Lemma,POS,Sent_ids,Ids
0,в,в,PREP,0 0 0 1 2 2 4 5 6 6 7 7 8 8 10 20 20 26 26 27 ...,"[2053, 2598, 1373, 1700, 316, 63, 3340, 3281, ..."
1,июне,июнь,NOUN,0 861,"[0, 861]"
2,этого,это,NPRO,0 5 30 41 41 88 146 192 197 210 326 335 363 40...,"[2336, 504, 5, 928, 1829, 3156, 1489, 146, 175..."
3,года,год,NOUN,0 260 261 269 432 447 473 505 552 556 699 736 ...,"[1171, 2553, 2461, 2325, 1570, 1272, 1490, 260..."
4,девушка,девушка,NOUN,0 244 250 269 450 905 909 925 1000 1241 1303 1...,"[2375, 1746, 2282, 1303, 1407, 3001, 1839, 310..."
...,...,...,...,...,...
15527,систем,система,NOUN,3465 3466,"[3466, 3465]"
15528,монтирует,монтировать,VERB,3466,[3466]
15529,оборудование,оборудование,NOUN,3466,[3466]
15530,устанавливает,устанавливать,VERB,3466,[3466]


In [ ]:
# Для скачивания tsv с униграммами
from google.colab import files
uni_df.to_csv('uni_df.tsv', sep='\t', index=False)
files.download('uni_df.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Датафрейм биграмм


In [ ]:
bi_df = pd.DataFrame(columns=['bi_token', 'uni_first_token', 'uni_first_lemma', 'uni_first_POS', 'uni_second_token', 'uni_second_lemma', 'uni_second_POS', 'Sent_ids'])
token_values = []
for sent_id, sent in tqdm(zip(sent_df.index, sent_df.Clean_sentence)):
  list_words = [word.strip(punctuation) for word in sent.split()]
  text_list = [' '.join(list_words)]
  bigrams = [b for l in text_list for b in zip(l.split(" ")[:-1], l.split(" ")[1:])]
  for bigram in bigrams:
    if bigram not in token_values:
      bi_lem = [get_pos_lem(word) for word in bigram]
      bi_df = bi_df.append({'bi_token': bigram,
                          'uni_first_token': bigram[0],
                          'uni_first_lemma': bi_lem[0][0],
                          'uni_first_POS': bi_lem[0][1],
                          'uni_second_token': bigram[1],
                          'uni_second_lemma': bi_lem[1][0], 
                          'uni_second_POS': bi_lem[1][1],
                          'Sent_ids': str(sent_id)
                          }, ignore_index=True)
      token_values.append(bigram)
    else:
      bi_df.loc[bi_df.bi_token == bigram, 'Sent_ids'] += f' {sent_id}'
   

In [ ]:
bi_df['Ids'] = bi_df['Sent_ids'].apply(fix_ids)

In [ ]:
bi_df

,bi_token,uni_first_token,uni_first_lemma,uni_first_POS,uni_second_token,uni_second_lemma,uni_second_POS,Sent_ids,Ids
0,"(в, июне)",в,в,PREP,июне,июнь,NOUN,0 861,"[861, 0]"
1,"(июне, этого)",июне,июнь,NOUN,этого,это,NPRO,0,[0]
2,"(этого, года)",этого,это,NPRO,года,год,NOUN,0 473 1169 1672 1756 3155,"[1672, 3155, 1169, 473, 1756, 0]"
3,"(года, девушка)",года,год,NOUN,девушка,девушка,NOUN,0,[0]
4,"(девушка, по)",девушка,девушка,NOUN,по,по,PREP,0,[0]
...,...,...,...,...,...,...,...,...,...
41739,"(устанавливает, датчики)",устанавливает,устанавливать,VERB,датчики,датчик,NOUN,3466,[3466]
41740,"(датчики, и)",датчики,датчик,NOUN,и,и,CONJ,3466,[3466]
41741,"(затем, программирует)",затем,затем,ADVB,программирует,программировать,VERB,3466,[3466]
41742,"(программирует, их)",программирует,программировать,VERB,их,они,NPRO,3466,[3466]


In [ ]:
# Для скачивания tsv с биграммами
from google.colab import files
bi_df.to_csv('bi_df.tsv', sep='\t', index=False)
files.download('bi_df.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Датафрейм триграмм

In [ ]:
tri_df = pd.DataFrame(columns=['tri_token', 'uni_first_token', 'uni_first_lemma', 'uni_first_POS', 'uni_second_token', 'uni_second_lemma', 'uni_second_POS','uni_third_token','uni_third_lemma','uni_third_POS', 'Sent_ids'])
token_values = []
for sent_id, sent in tqdm(zip(sent_df.index, sent_df.Clean_sentence)):
    list_words = [word.strip(punctuation) for word in sent.split()]
    text_list = [' '.join(list_words)]
    trigrams = [b for l in text_list for b in zip(l.split(" ")[:-1], l.split(" ")[1:],l.split(' ')[2:])]
    for trigram in trigrams:
        if trigram not in token_values:
            tri_lem = [get_pos_lem(word) for word in trigram]
            tri_df = tri_df.append({'tri_token': trigram,
                                'uni_first_token': trigram[0],
                                'uni_first_lemma': tri_lem[0][0],
                                'uni_first_POS': tri_lem[0][1],
                                'uni_second_token': trigram[1],
                                'uni_second_lemma': tri_lem[1][0], 
                                'uni_second_POS': tri_lem[1][1],
                                'uni_third_token': trigram[2],
                                'uni_third_lemma': tri_lem[2][0], 
                                'uni_third_POS': tri_lem[2][1],
                                'Sent_ids': str(sent_id)
                          }, ignore_index=True)
            token_values.append(trigram)
        else:
            tri_df.loc[tri_df.tri_token == trigram, 'Sent_ids'] += f' {sent_id}'

In [ ]:
tri_df['Ids'] = tri_df['Sent_ids'].apply(fix_ids)

In [ ]:
tri_df

,tri_token,uni_first_token,uni_first_lemma,uni_first_POS,uni_second_token,uni_second_lemma,uni_second_POS,uni_third_token,uni_third_lemma,uni_third_POS,Sent_ids,Ids
0,"(в, июне, этого)",в,в,PREP,июне,июнь,NOUN,этого,это,NPRO,0,[0]
1,"(июне, этого, года)",июне,июнь,NOUN,этого,это,NPRO,года,год,NOUN,0,[0]
2,"(этого, года, девушка)",этого,это,NPRO,года,год,NOUN,девушка,девушка,NOUN,0,[0]
3,"(года, девушка, по)",года,год,NOUN,девушка,девушка,NOUN,по,по,PREP,0,[0]
4,"(девушка, по, имени)",девушка,девушка,NOUN,по,по,PREP,имени,имя,NOUN,0,[0]
...,...,...,...,...,...,...,...,...,...,...,...,...
46753,"(устанавливает, датчики, и)",устанавливает,устанавливать,VERB,датчики,датчик,NOUN,и,и,CONJ,3466,[3466]
46754,"(датчики, и, затем)",датчики,датчик,NOUN,и,и,CONJ,затем,затем,ADVB,3466,[3466]
46755,"(и, затем, программирует)",и,и,CONJ,затем,затем,ADVB,программирует,программировать,VERB,3466,[3466]
46756,"(затем, программирует, их)",затем,затем,ADVB,программирует,программировать,VERB,их,они,NPRO,3466,[3466]


In [ ]:
# Для скачивания tsv с триграммами
from google.colab import files
tri_df.to_csv('tri_df.tsv', sep='\t', index=False)
files.download('tri_df.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
uni_df = pd.read_csv('https://raw.githubusercontent.com/tbkazakova/autobr_femcorpus/main/uni_df.tsv', sep='\t', header=0, names=['Token1', 'Lemma1', 'POS1', 'Sent_ids'], index_col=False) 

In [ ]:
bi_df = pd.read_csv('https://raw.githubusercontent.com/tbkazakova/autobr_femcorpus/main/bi_df.tsv', sep='\t', header=0, names=['bi_token', 'Token1', 'Lemma1', 'POS1', 'Token2', 'Lemma2', 'POS2', 'Sent_ids'], index_col=False) 

In [ ]:
tri_df = pd.read_csv('https://raw.githubusercontent.com/tbkazakova/autobr_femcorpus/main/tri_df.tsv', sep='\t', header=0, names=['tri_token', 'Token1', 'Lemma1', 'POS1', 'Token2', 'Lemma2', 'POS2', 'Token3', 'Lemma3', 'POS3', 'Sent_ids'], index_col=False) 

## ПОИСК

Вариант без выделения искомого. Выдача - таблица.

In [180]:
POStags = ['NOUN', 'ADJF', 'ADJS', 'COMP', 'VERB', 'INFN', 'PRTF', 'PRTS', 'GRND', 'NUMR', 'ADVB', 'NPRO', 'PRED', 'PREP', 'CONJ', 'PRCL', 'INTJ', 'None']
trash_symbols = '!"#$%&\'-()*+,./:;<=>?@[\\]^_`{|}~«»—'

def make_condition(req, index):
    if '+' in req:
        req = req.split('+')
        req[0] = req[0].strip('"').lower()
        req[1] = req[1].strip('"')
        cond = f'Token{index} == "{req[0]}" and POS{index} == "{req[1]}"'
    elif req.startswith('"') and req.endswith('"'):
        req = req.lower()
        req = req.replace('"', '')
        cond = f'Token{index} == "{req}"'
    elif req in POStags:
        cond = f'POS{index} == "{req}"'
    else:
        req = req.strip(trash_symbols).lower()
        req = morph.parse(req)[0].normal_form
        cond = f'Lemma{index} == "{req}"'
    return cond

def get_sentences(req, df):
    conditions = []
    for i, w in enumerate(req.split()):
        conditions.append(make_condition(w, i+1))
    condition = ' and '.join(conditions)
    print(condition)
    sent_ids = fix_ids(' '.join(df.query(condition)['Sent_ids'].values.tolist()))
    sentences = sent_df.loc[sent_ids]
    
    
    return sentences

In [181]:
def get_answer(req):
    if len(req.split()) == 1:
        res = get_sentences(req, uni_df)
    elif len(req.split()) == 2:
        res = get_sentences(req, bi_df)
    elif len(req.split()) == 3:
        res = get_sentences(req, tri_df)
    else:
        print('Запрос в неправильном формате.')
        
    return res

In [182]:
get_answer('"в" ADJF время') 

Token1 == "в" and POS2 == "ADJF" and Lemma3 == "время"


,URL,Date,Sentence,Clean_sentence
1391,https://vk.com/public46661081?w=wall-46661081_...,2020-10-14 17:30:02,В ООН в настоящее время входит 193 государства.,в оон в настоящее время входит государства
2636,https://vk.com/public46661081?w=wall-46661081_...,2020-09-28 17:30:01,Ведущий в элегантной манере представил его как...,ведущий в элегантной манере представил его как...
290,https://vk.com/public182023900?w=wall-18202390...,2019-12-18 14:00:03,"В настоящее время женщины могут делать, что он...",в настоящее время женщины могут делать что они...
371,https://vk.com/public182023900?w=wall-18202390...,2019-10-04 19:41:56,"Итак, устранение бинарности пола в последнее в...",итак устранение бинарности пола в последнее вр...
590,https://vk.com/public182023900?w=wall-18202390...,2019-07-06 07:00:05,"И её беспокоят современные тенденции, из-за ко...",и её беспокоят современные тенденции из-за кот...
278,https://vk.com/public182023900?w=wall-18202390...,2019-12-23 15:01:15,«Почему вопрос пола никогда не был таким сложн...,почему вопрос пола никогда не был таким сложны...
506,https://vk.com/public182023900?w=wall-18202390...,2019-07-30 17:00:09,"В растущей алфавитной каше, в нынешнее время с...",в растущей алфавитной каше в нынешнее время со...
611,https://vk.com/public182023900?w=wall-18202390...,2019-06-21 09:06:02,В настоящее время я беседую со многими молодым...,в настоящее время я беседую со многими молодым...
26,https://vk.com/public182023900?w=wall-18202390...,2020-09-23 16:13:27,Вот почему цивилизация полностью разрушилась в...,вот почему цивилизация полностью разрушилась в...
2368,https://vk.com/public46661081?w=wall-46661081_...,2020-10-03 17:30:01,Одного из подозреваемых около 20 лет назад осу...,одного из подозреваемых около лет назад осудил...


## Вариант с выделением **искомого**

In [ ]:
class bcolors:
    CBLACK  = '\33[30m'
    FAIL = '\033[91m'
  

In [ ]:
def light_up_print(ind, line, sentence, url):
  if sentence[0] == ' ':
    sent = ' ' + sentence[1].lower() + sentence[2:]
  else:
    sent = ' ' + sentence[0].lower() + sentence[1:]
  reg = re.compile(f'([^а-яА-Я]{line}[^а-яА-Я])')
  l = re.split(reg, sent)
  print(bcolors.CBLACK + ind, end=' ')
  for i in range(len(l)):
    if (i%2) == 0:
      if i == 0 and l[0] != '':
        print(bcolors.CBLACK + l[i][1].capitalize(), bcolors.CBLACK + l[i][2:], sep='', end='')
      else:
        print(bcolors.CBLACK + l[i], end='')
    else:
      if i == 1  and l[0] == '':
        print(bcolors.FAIL + l[i][1].capitalize(), bcolors.FAIL + l[i][2:], sep='', end='')
      else:  
        print(bcolors.FAIL + l[i], end='')
  print(bcolors.CBLACK +'\n(', bcolors.CBLACK + url, bcolors.CBLACK + ')')
    
    

In [183]:
POStags = ['NOUN', 'ADJF', 'ADJS', 'COMP', 'VERB', 'INFN', 'PRTF', 'PRTS', 'GRND', 'NUMR', 'ADVB', 'NPRO', 'PRED', 'PREP', 'CONJ', 'PRCL', 'INTJ', 'None']
trash_symbols = '!"#$%&\'-()*+,./:;<=>?@[\\]^_`{|}~«»—'

def make_condition(req, index):
    if '+' in req:
        req = req.split('+')
        req[0] = req[0].strip('"').lower()
        req[1] = req[1].strip('"')
        cond = f'Token{index} == "{req[0]}" and POS{index} == "{req[1]}"'
    elif req.startswith('"') and req.endswith('"'):
        req = req.lower()
        req = req.replace('"', '')
        cond = f'Token{index} == "{req}"'
    elif req in POStags:
        cond = f'POS{index} == "{req}"'
    else:
        req = req.strip(trash_symbols).lower()
        req = morph.parse(req)[0].normal_form
        cond = f'Lemma{index} == "{req}"'
    return cond

def get_sentences(req, df):
    conditions = []
    lines = []
    for i, w in enumerate(req.split()):
        cond = make_condition(w, i+1)
        conditions.append(cond)

    condition = ' and '.join(conditions)
    tokens = df.query(condition)
    
    if i == 2:
      size = 'tri_token'
    if i == 1:
      size = 'bi_token'
    if i == 0:
      size = 'Token1'
    for el in tokens[size].values.tolist():
      reg = re.compile('[а-яА-Я0-9_]+')
      a = reg.findall(el)
      s = ' '.join(a)
      lines.append(s)
    sent_ids = df.query(condition)['Sent_ids']
    if len(tokens[size].values.tolist()) == 0:
      print('По вашему запросу ничего не нашлось.')
    else:
      for i, line in zip(sent_ids, lines):
        for ind in i.split():
          light_up_print(ind, line, ' ' + sent_df.loc[int(ind)]['Sentence'], sent_df.loc[int(ind)]['URL'])
    

In [184]:
def get_answer():
    req = input('Введите ваш запрос: ')
    if len(req.split()) == 1:
        get_sentences(req, uni_df)
    elif len(req.split()) == 2:
        get_sentences(req, bi_df)
    elif len(req.split()) == 3:
        get_sentences(req, tri_df)
    else:
        print('Запрос в неправильном формате.')
    

In [193]:
get_answer()

Введите ваш запрос: NUMR женщина
536 Двадцать две женщины подали в суд на компанию по производству порнографии за мошенничество, принуждение и введение в заблуждение  Согласно материалам дела, «Girls Do Porn» размещали объявления на площадке Craigslist о подработке моделью, но заинтересовавшиеся объявлением женщины выясняли, что на самом деле компания собирается снимать порно.
( https://vk.com/public182023900?w=wall-182023900_145 )
641 Обе женщины вскоре были доставлены в больницу с повреждениями лица, но на данный момент их уже выписали.
( https://vk.com/public182023900?w=wall-182023900_34 )
3321 «Очевидно, что нарушены конституционные права женщин на личную неприкосновенность»  Юрист АНО  Синяя Птица  Ольга Неклюдова рассказала NewsVo, что обе женщины, страдающие от преследований бывших мужей, домашних тиранов, обратились в их организацию летом 2020 года.
( https://vk.com/public46661081?w=wall-46661081_76215 )
642 Старший детектив Энди Кокс прокомментировал сложившуюся ситуацию так: 